In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models
from tensorflow.keras.saving import register_keras_serializable
import tensorflow as tf
import tf2onnx
import joblib
import json
data = pd.read_csv("dataset/earthquake_data.csv")
X = data.drop("quake_binary", axis=1).astype("float32")

def generate_trust_score(row):
    moment = row["seismic_moment_rate"]
    disp   = row["surface_displacement_rate"]
    stress = row["coulomb_stress_change"]
    depth  = row["average_focal_depth"]
    slip   = row["fault_slip_rate"]

    moment_factor = np.clip((moment - 5e15) / 1.5e16, 0, 1)
    disp_factor   = np.clip((disp - 10) / 40, 0, 1)
    stress_factor = np.clip((stress - 100) / 500, 0, 1)
    depth_factor  = np.clip((30 - depth) / 30, 0, 1)     # Shallow quakes = higher trust
    slip_factor   = np.clip((slip - 2) / 10, 0, 1)

    composite = (0.25 * moment_factor +
                 0.2  * disp_factor +
                 0.2  * stress_factor +
                 0.2  * depth_factor +
                 0.15 * slip_factor)

    return 0.5 + composite

trust_labels = X.apply(generate_trust_score, axis=1).astype("float32")

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
joblib.dump(scaler, "models/QuakeTrustScaler.pkl")

@register_keras_serializable()
def trust_activation(x):
    return 0.5 + tf.sigmoid(x)

input_layer = layers.Input(shape=(5,))
z = layers.Dense(32, activation="relu")(input_layer)
z = layers.Dense(16, activation="relu")(z)
z = layers.Dense(8, activation="relu")(z)
trust_output = layers.Dense(1, activation=trust_activation)(z)

model = models.Model(inputs=input_layer, outputs=trust_output)
model.compile(optimizer="adam", loss="mse")

model.fit(X_scaled, trust_labels, validation_split=0.2, batch_size=32, epochs=10)
model.summary()
model.save("models/QuakeTrustNet.h5")
model_proto, _ = tf2onnx.convert.from_keras(model)
with open(f"models/ONNX/QuakeTrustNet.onnx", "wb") as f:
    f.write(model_proto.SerializeToString())

scaler_data = {
        "mean": scaler.mean_.tolist(),
        "scale": scaler.scale_.tolist()
}
with open(f"models/SCALER-CFG/QuakeScaler.json", "w") as f:
    json.dump(scaler_data, f)

Epoch 1/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0558 - val_loss: 0.0021
Epoch 2/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0015 - val_loss: 8.7257e-04
Epoch 3/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 8.2590e-04 - val_loss: 6.2641e-04
Epoch 4/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 6.6322e-04 - val_loss: 4.9918e-04
Epoch 5/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 5.3829e-04 - val_loss: 4.3643e-04
Epoch 6/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 4.7965e-04 - val_loss: 4.2685e-04
Epoch 7/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 4.3754e-04 - val_loss: 3.6044e-04
Epoch 8/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 4.1148e-04 - val_loss: 3.4259e-04
Epoch 9/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 3.7874e-04 - val_loss: 3.3782e-04
Epoch 10/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 3.5104e-04 - val_loss: 3.1984e-04


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 5)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,597 (10.15 KB)

 Trainable params: 865 (3.38 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,732 (6.77 KB)

ERROR:tf2onnx.tfonnx:rewriter <function rewrite_constant_fold at 0x0000025F301FA660>: exception `np.cast` was removed in the NumPy 2.0 release. Use `np.asarray(arr, dtype=dtype)` instead.
